In [60]:
from xml.dom import minidom
import glob
import pandas as pd

notas = glob.glob (r"D:\Milton\arquivo\XML THIAGO\*.xml")
dados={'Produto':[],'Ean':[],'Ncm':[],'CEST':[],'CFOP':[], 'uCom':[], 'vProd':[] , 'CSOSN':[], 'CST':[], 'Aliquota':[], 'Tributacao':[]}
#Dicionário de dados recolhidos das notas juntamente com caminho da pasta das notas em formato XML


In [61]:
for n in notas:
    xml = open(n,encoding="utf-8")
    nfe=minidom.parse(xml)#PERCORRER AS NOTAS DA PASTA DO GLOB    
    itens = nfe.getElementsByTagName('det')    
    
    for i in itens:
        
        numitem=int(i.attributes['nItem'].value)-1
        
        ean = i.getElementsByTagName('cEAN')#COD DE BARRAS
        dados['Ean'].append(ean[0].firstChild.nodeValue)
        
        prod = i.getElementsByTagName('xProd')#PRODUTO
        dados['Produto'].append(prod[0].firstChild.nodeValue)
        
        ncm = i.getElementsByTagName ('NCM')#NCM
        dados['Ncm'].append(ncm[0].firstChild.nodeValue)
        
        
        cest = i.getElementsByTagName('CEST')
        if cest:
            dados['CEST'].append(cest[0].firstChild.nodeValue)#VERIFICAR SE EXISTE A TAG DE CEST
        else:    
            dados['CEST'].append('0000000')#PREENCHER COM ALGO CASO NAO EXISTA
        
    
        und = i.getElementsByTagName('uCom')#UNIDADE
        dados['uCom'].append(und[0].firstChild.nodeValue)
        
        
        vProd = i.getElementsByTagName('vProd')     
        if vProd:
            dados['vProd'].append(vProd[0].firstChild.nodeValue)#VERIFICAR SE EXISTE A TAG DE VALOR DO PRODUTO
        else:
            dados['vProd'].append('0')#PREENCHER COM ALGO CASO NAO EXISTA
                 
            
        cfop = i.getElementsByTagName('CFOP')      
        if cfop[0].firstChild.nodeValue in ('5403','5405','6403','6405'): #INSERINDO DADOS DE TRIBUTAÇÃO DE ACORDO COM CFOP
            dados['CFOP'].append('5405')#INSERE CFOP
            dados['CSOSN'].append('500')#INSERE CSOSN
            dados['CST'].append('060')#INSERE CST
            dados['Aliquota'].append('0')#INSERE ALIQUOTA
            dados['Tributacao'].append('1')#INSERE TRIBUTACAO (0 = ISENTO, 1 = SUB TRIBUTARIA, 2 = TRIBUTADO)

        elif cfop[0].firstChild.nodeValue in ('5102','6102','6101','5101'):
            dados['CFOP'].append('5102')#INSERE CFOP
            dados['CSOSN'].append('102')#INSERE CSOSN
            dados['CST'].append('000')#INSERE CST
            dados['Aliquota'].append('18')#INSERE ALIQUOTA
            dados['Tributacao'].append('2')#INSERE TRIBUTACAO (0 = ISENTO, 1 = SUB TRIBUTARIA, 2 = TRIBUTADO)
            
        else:
            dados['CFOP'].append('5405')#INSERE CFOP
            dados['CSOSN'].append('500')#INSERE CSOSN
            dados['CST'].append('060')#INSERE CST
            dados['Aliquota'].append('0')#INSERE ALIQUOTA
            dados['Tributacao'].append('1')#INSERE TRIBUTACAO (0 = ISENTO, 1 = SUB TRIBUTARIA, 2 = TRIBUTADO)
            
        
  

In [62]:
df = pd.DataFrame(dados)#Inserindo os dados das notas no dataframe
df = df.sort_values('Produto', ascending=True)#Ordenando dataframe
df = df.drop_duplicates(subset='Produto', keep='first')#Removendo produtos com descricao duplicadas
df = df.drop_duplicates(subset='Ean', keep='first')#Removendo produtos com código de barras duplicadas
df = df.reset_index(drop=True)#Resetando o indice e dropando o antigo


In [64]:
df.to_csv(r"D:\Milton\arquivo\XML THIAGO\thiago teste.csv",sep=';')#Salvando o dataframe no arquivo csv